
![](http://cloud.millenniumpost.in/3852j9Wg4JQzOfJCYmAVLqrN5kPrjxtniGix1153101)

# Analysis of Suicide Rates by Country and Sex

- <a href='#1'>1. Introduction</a>  
- <a href='#2'>2. Retrieving the Data</a>
     - <a href='#2.1'>2.1 Importing Libraries</a>
     - <a href='#2.2'>2.2 Loading the data</a>
- <a href='#3'>3. Data Preparation</a>
     - <a href='#3.1'>3.1 Merging the first two databases</a>
     - <a href='#3.2'>3.2 Transforming data of the third database</a>
     - <a href='#3.3'>3.3 Filling uniform data and fixing wrong data</a>     
- <a href='#4'>4. Overview of the Data</a>
     - <a href='#4.1'>4.1 Overview of the main database</a>
     - <a href='#4.2'>4.2 Overview of the merged database</a>
- <a href='#5'>5. Data Exploration</a>
    - <a href='#5.1'>5.1 Top countries with the highest suicide rates</a>
    - <a href='#5.2'>5.2 Top countries with the highest suicide rates for males</a>
    - <a href='#5.3'>5.3 Top countries with the highest suicide rates for females</a>
    - <a href='#5.4'>5.4 Top countries with the highest suicide rates by years</a>
        - <a href='#5.4.1'>5.4.1 Top countries with highest suicide rate in one year</a>
        - <a href='#5.4.2'>5.4.2 Highest Suicide Rates in 2015</a>
        - <a href='#5.4.3'>5.4.3 Top countries with the highet suicide rates average since the 90's</a>
    - <a href='#5.5'>5.5 Top countries with the highest or lowest male-female ratio of suicide rates</a>
    - <a href='#5.6'>5.6 Further analysis on particular countries</a>   
- <a href='#6'>6. Summary and Conculsions</a>
    - <a href='#6.1'>6.1 Summary of the data presented in the notebook</a>
    - <a href='#6.2'>6.2 Conclusions</a>

# <a id='1'> 1. Introduction</a>

Close to 800 000 people die due to suicide every year, which is one person every 40 seconds. Many more attempt suicide. Suicide occurs throughout the lifespan and is the second leading cause of death among 15-29 year olds globally.

Suicide is a global phenomenon; in fact, 78% of suicides occurred in low- and middle-income countries in 2015. Suicide accounted for 1.4% of all deaths worldwide, making it the 17th leading cause of death in 2015. Effective and evidence-based interventions can be implemented at population, sub-population and individual levels to prevent suicide and suicide attempts.

There are problems with availablity and accuracy of data about suicides. The data that will be explored and described in this notebook was obtained from several sources:

• World Health Organization (WHO) dataset which contains entity wise suicide rates, crude suicide rates per gender and country which are age standardized which has a geographical coverage of 198 countries. The time spanning from 1950-2011.

• Samaritans statistics report 2017 including data for 2013-2015, in order to reduce the time, it takes to register deaths, the maximum time between a death and registration is eight days.

• American Association of Suicidology facts and statistics which are categorized by age, gender, region and ethnicity.

The goal of this data exploration process is to visualize the trends and patterns by merging different datasets available regarding the subject matter from different organizations, and observing the changes in patterns over the years by country, sex and ethnicity.

# <a id='2'>2. Retrieving the Data</a>

## <a id='2.1'>2.1 Importing Libraries</a>

In [ ]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
import warnings
warnings.filterwarnings('ignore') # Shuts down irrelevant userWarnings

In [ ]:
import plotly.offline as py
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
import plotly.offline as offline
import plotly.graph_objs as go

## <a id='2.2'>2.2 Loading the data</a>

In [ ]:
sui_sex_ratios = pd.read_csv("../input/Male-Female-Ratio-of-Suicide-Rates.csv").sort_values(by='Entity', ascending=True)

In [ ]:
sui_both = pd.read_csv("../input/suicide-rates-by-country.csv").sort_values(by='Entity', ascending=True)

In [ ]:
sui_all = pd.read_csv("../input/SDGSUICIDE.csv")

# <a id='3'>3. Data Preparation</a>

## <a id='3.1'>3.1 Merging the first two databases</a>

In [ ]:
# Generatina a merged dataset
suicides_merge = pd.merge(sui_sex_ratios, sui_both, how='inner').sort_values(by='Entity', ascending=True)
suicides_merge.columns = ['Country', 'Code', 'Year', 'Male-Female ratio', 'Total suicide rate']
suicides_merge = suicides_merge.dropna()

## <a id='3.2'>3.2 Transforming data of the third database</a>

In [ ]:
sui_all.columns = sui_all.iloc[0].values
for i in range (2, 6):
    sui_all.columns.values[i] = str(int(sui_all.columns[i]))


sui_all = sui_all.drop(0)
sui_all.reset_index(drop = True, inplace = True)

In [ ]:
suicides = pd.DataFrame(columns=['Country', 'Male suicide rate', 'Female suicide rate', 'Total suicide rate', 'Year'])
Years = ['2000', '2005', '2010', '2015']

deleted = []
                  
for year in Years:
    both_df = sui_all[sui_all['Sex'].str.contains("Both sexes")][['Country', year]]
    male_df = sui_all[sui_all['Sex'].str.contains("Male")][['Country', year]]
    female_df = sui_all[sui_all['Sex'].str.contains("Female")][['Country', year]]

    both_df.reset_index(drop = True, inplace = True)
    male_df.reset_index(drop = True, inplace = True)
    female_df.reset_index(drop = True, inplace = True)
    country_df = both_df['Country']

    year_df = pd.concat([male_df,female_df,both_df], axis=1)
    del year_df['Country'] # To get a dataframe without multiple 'Country' columns

    year_df = pd.concat([country_df,year_df], axis=1)
    
    year_df['Year'] = year # To get a dataframe that includes Country and Year
    year_df.columns = ['Country', 'Male suicide rate', 'Female suicide rate', 'Total suicide rate', 'Year']
    year_df.replace([0, np.inf], np.nan)
    year_df = year_df.dropna()
    year_df['Male-Female ratio'] = year_df['Male suicide rate'] / year_df['Female suicide rate']
    
    suicides = pd.concat([suicides,year_df])
suicides.reset_index(drop = True, inplace = True)

## <a id='3.3'>3.3 Filling uniform data and fixing wrong data</a>

In [ ]:
suicides = suicides.replace('Republic of Korea', 'South Korea')
suicides = suicides.replace('Russian Federation', 'Russia')
suicides = suicides.replace('Bolivia (Plurinational State of)', 'Bolivia')
suicides = suicides.replace('Democratic People\'s Republic of Korea', 'North Korea')
suicides = suicides.replace('Republic of Moldova', 'Moldova')
suicides = suicides.replace('United States of America', 'United States')
suicides = suicides.replace('Czechia', 'Czech Republic')
suicides = suicides.replace('Lao People\'s Democratic Republic', 'Laos')
suicides = suicides.replace('Micronesia (Federated States of)', 'Micronesia')
suicides = suicides.replace('United Kingdom of Great Britain and Northern Ireland', 'United Kingdom')
suicides = suicides.replace('The former Yugoslav republic of Macedonia', 'Macedonia')
suicides = suicides.replace('Viet Nam', 'Vietnam')
suicides = suicides.replace('United Republic of Tanzania', 'Tanzania')
suicides = suicides.replace('Iran (Islamic Republic of)', 'Iran')
suicides = suicides.replace('Venezuela (Bolivarian Republic of)', 'Venezuela')
suicides = suicides.replace('Syrian Arab Republic', 'Syria')
suicides = suicides.replace('Brunei Darussalam', 'Brunei')
suicides = suicides.replace('Cabo Verde', 'Cape Verde')

In [ ]:
suicides_merge = suicides_merge.replace('Democratic Republic of Congo', 'Democratic Republic of the Congo')
suicides_merge = suicides_merge.replace('Timor', 'Timor-Leste')
suicides_merge = suicides_merge.replace('Micronesia (country)', 'Micronesia')

In [ ]:
countriesCodes = dict(zip(suicides_merge['Country'].tolist(), suicides_merge['Code'].tolist()))
countriesNames = dict(zip(suicides_merge['Code'].tolist(), suicides_merge['Country'].tolist()))

In [ ]:
newCodes = {'South Sudan':'SSD', 'Kiribati':'KIR', 'Montenegro':'MNE', 'Côte d\'Ivoire':'CIV'}
newNames = {'SSD':'South Sudan', 'KIR':'Kiribati', 'MNE':'Montenegro', 'CIV':'Côte d\'Ivoire'}

In [ ]:
countriesCodes = dict(countriesCodes, **newCodes)
countriesNames = dict(countriesNames, **newNames)

# <a id='4'>4. Overview of the Data</a>

## <a id='4.1'>4.1 Overview of the main database</a>

In [ ]:
print("Size of the database:", suicides.shape)

In [ ]:
suicides.head()

**Statistical information about the data**

In [ ]:
suicides.describe()

**Information about the data types**

In [ ]:
suicides.info()

## <a id='4.2'>4.2 Overview of the merged database</a>

In [ ]:
print("Size of the merged database:", suicides_merge.shape)

In [ ]:
suicides_merge.head()

**Statistical information about the data**

In [ ]:
suicides_merge.describe()

**Information about the data types**

In [ ]:
suicides_merge.info()

# <a id='5'>5. Data Exploration</a>

In [ ]:
mask2000 = suicides['Year'].astype(int) == 2000
mask2005 = suicides['Year'].astype(int) == 2005
mask2010 = suicides['Year'].astype(int) == 2010
mask2015 = suicides['Year'].astype(int) == 2015
maskSince1990 = suicides_merge['Year'].astype(int) >= 1990

## <a id='5.1'>5.1 Top countries with the highest suicide rates</a>

In [ ]:
total2000 = suicides[mask2000][['Country', 'Total suicide rate']].sort_values(by='Total suicide rate', ascending=False)
total2005 = suicides[mask2005][['Country', 'Total suicide rate']].sort_values(by='Total suicide rate', ascending=False)
total2010 = suicides[mask2010][['Country', 'Total suicide rate']].sort_values(by='Total suicide rate', ascending=False)
total2015 = suicides[mask2015][['Country', 'Total suicide rate']].sort_values(by='Total suicide rate', ascending=False)

In [ ]:
t2000 = go.Bar(
    x = total2000[total2000['Country'].isin(total2015['Country'].head(10))]['Country'].head(10),
    y = total2000[total2000['Country'].isin(total2015['Country'].head(10))]['Total suicide rate'].head(10),
    marker=dict(color='rgb(129, 67, 116)'),
    name = '2000'
)

t2005 = go.Bar(
    x = total2005[total2005['Country'].isin(total2015['Country'].head(10))]['Country'].head(10),
    y = total2005[total2005['Country'].isin(total2015['Country'].head(10))]['Total suicide rate'].head(10),
    marker=dict(color='rgb(81, 163, 157)'),
    name = '2005'
)

t2010 = go.Bar(
    x = total2010[total2010['Country'].isin(total2015['Country'].head(10))]['Country'].head(10),
    y = total2010[total2010['Country'].isin(total2015['Country'].head(10))]['Total suicide rate'].head(10),
    marker=dict(color='rgb( 183, 105, 92)'),
    name = '2010'
)

t2015 = go.Bar(
    x = total2015['Country'].head(10),
    y = total2015['Total suicide rate'].head(10),
    marker=dict(color='rgb(205, 187, 121)',
    ),
    name = '2015'
)


data = [t2000, t2005, t2010, t2015]

layout = go.Layout(
    title='Top countries with the highest suicide rates for both sexes',
    xaxis={'title':'Countries'},
    yaxis={'title':'Suicide rate (per 100,000 people)'},
    barmode='group',
    bargap=0.15,
    bargroupgap=0.1
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='basic-bar')

**Top 10 countries with the highest suicide rates for both sexes:**

*(Top 10 in 2015, although data is shown for 2000, 2005, 2010 and 2015 and ordered by the data from 2000)*

* Lithuania
* Belarus
* Kazakhstan
* Sri Lanka
* Mongolia
* Suriname
* Equatorial Guinea
* Guyana
* Poland
* South Korea

## <a id='5.2'>5.2 Top countries with the highest suicide rates for males</a>

In [ ]:
male2000 = suicides[mask2000][['Country', 'Male suicide rate']].sort_values(by='Male suicide rate', ascending=False)
male2005 = suicides[mask2005][['Country', 'Male suicide rate']].sort_values(by='Male suicide rate', ascending=False)
male2010 = suicides[mask2010][['Country', 'Male suicide rate']].sort_values(by='Male suicide rate', ascending=False)
male2015 = suicides[mask2015][['Country', 'Male suicide rate']].sort_values(by='Male suicide rate', ascending=False)

In [ ]:
t2000 = go.Bar(
    x = male2000[male2000['Country'].isin(male2015['Country'].head(10))]['Country'].head(10),
    y = male2000[male2000['Country'].isin(male2015['Country'].head(10))]['Male suicide rate'].head(10),
    marker=dict(color='rgb(55, 83, 109)'),
    name = '2000'
)

t2005 = go.Bar(
    x = male2005[male2005['Country'].isin(male2015['Country'].head(10))]['Country'].head(10),
    y = male2005[male2005['Country'].isin(male2015['Country'].head(10))]['Male suicide rate'].head(10),
    marker=dict(color='rgb(133, 133, 173)'),
    name = '2005'
)

t2010 = go.Bar(
    x = male2010[male2010['Country'].isin(male2015['Country'].head(10))]['Country'].head(10),
    y = male2010[male2010['Country'].isin(male2015['Country'].head(10))]['Male suicide rate'].head(10),
    marker=dict(color='rgb(101, 190, 190)'),
    name = '2010'
)

t2015 = go.Bar(
    x = male2015['Country'].head(10),
    y = male2015['Male suicide rate'].head(10),
    marker=dict(color='rgb(26, 118, 255)'),
    name = '2015'
)

data = [t2000, t2005, t2010, t2015]

layout = go.Layout(
    title='Top countries with the highest suicide rates for males',
    xaxis={'title':'Countries'},
    yaxis={'title':'Suicide rate (per 100,000 people)'},
    barmode='group',
    bargap=0.15,
    bargroupgap=0.1
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='basic-bar')

**Top 10 countries with the highest suicide rates for males:**

*(Top 10 in 2015, although data is shown for 2000, 2005, 2010 and 2015 and ordered by the data from 2000)*

* Lithuania
* Belarus
* Kazakhstan
* Latvia
* Sri Lanka
* Mongolia
* Suriname
* Poland
* Guyana
* South Korea

## <a id='5.3'>5.3 Top countries with the highest suicide rates for females</a>

In [ ]:
female2000 = suicides[mask2000][['Country', 'Female suicide rate']].sort_values(by='Female suicide rate', ascending=False)
female2005 = suicides[mask2005][['Country', 'Female suicide rate']].sort_values(by='Female suicide rate', ascending=False)
female2010 = suicides[mask2010][['Country', 'Female suicide rate']].sort_values(by='Female suicide rate', ascending=False)
female2015 = suicides[mask2015][['Country', 'Female suicide rate']].sort_values(by='Female suicide rate', ascending=False)

In [ ]:
t2000 = go.Bar(
    x = female2000[female2000['Country'].isin(female2015['Country'].head(10))]['Country'].head(10),
    y = female2000[female2000['Country'].isin(female2015['Country'].head(10))]['Female suicide rate'].head(10),
    marker=dict(color='rgb(170, 0, 120)'),
    name = '2000'
)

t2005 = go.Bar(
    x = female2005[female2005['Country'].isin(female2015['Country'].head(10))]['Country'].head(10),
    y = female2005[female2005['Country'].isin(female2015['Country'].head(10))]['Female suicide rate'].head(10),
    marker=dict(color='rgb(198, 141, 227)'),
    name = '2005'
)

t2010 = go.Bar(
    x = female2010[female2010['Country'].isin(female2015['Country'].head(10))]['Country'].head(10),
    y = female2010[female2010['Country'].isin(female2015['Country'].head(10))]['Female suicide rate'].head(10),
    marker=dict(color='rgb(94, 154, 242)'),
    name = '2010'
)

t2015 = go.Bar(
    x = female2015['Country'].head(10),
    y = female2015['Female suicide rate'].head(10),
    marker=dict(color='rgb(0, 0, 120)'),
    name = '2015'
)

data = [t2000, t2005, t2010, t2015]

layout = go.Layout(
    title='Top countries with the highest suicide rates for females',
    xaxis={'title':'Countries'},
    yaxis={'title':'Suicide rate (per 100,000 people)'},
    barmode='group',
    bargap=0.15,
    bargroupgap=0.1
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='basic-bar')

**Top 10 countries with the highest suicide rates for females:**

*(Top 10 in 2015, although data is shown for 2000, 2005, 2010 and 2015 and ordered by the data from 2000)*

* Sri Lanka
* North Korea
* India
* Bolivia
* Japan
* Belguim
* Guyana
* Suriname
* Angola
* South Korea

These are slightly different countries than those we've seen for males and for both sexes summed up.

## <a id='5.4'>5.4 Top countries with the highest suicide rates by years</a>

In [ ]:
rate2000 = suicides[mask2000].sort_values(by='Total suicide rate', ascending=False)
rate2005 = suicides[mask2005].sort_values(by='Total suicide rate', ascending=False)
rate2010 = suicides[mask2010].sort_values(by='Total suicide rate', ascending=False)
rate2015 = suicides[mask2015].sort_values(by='Total suicide rate', ascending=False)

### <a id='5.4.1'>5.4.1 Top countries with highest suicide rate in one year</a>

**Countries with highest suicide rates in 2015**

In [ ]:
total = go.Bar(
    x = rate2015['Country'].head(15),
    y = rate2015['Total suicide rate'].head(15),
    marker=dict(color='rgb(53, 183, 85)'),
    name = 'Both Sexes'
)

male = go.Bar(
    x = rate2015['Country'].head(15),
    y = rate2015['Male suicide rate'].head(15),
    marker=dict(color='rgb(26, 118, 255)'),
    name = 'Male'
)

female = go.Bar(
    x = rate2015['Country'].head(15),
    y = rate2015['Female suicide rate'].head(15),
    marker=dict(color='rgb(255, 25, 64)'),
    name = 'Female'
)

data = [total, male, female]

layout = go.Layout(
    title='Top countries with the highest suicide rates in 2015',
    xaxis={'title':'Countries'},
    yaxis={'title':'Suicide rate (per 100,000 people)'},
    barmode='group',
    bargap=0.15,
    bargroupgap=0.1
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='basic-bar')

**Countries with highest suicide rates in 2010**

In [ ]:
total = go.Bar(
    x = rate2010['Country'].head(15),
    y = rate2010['Total suicide rate'].head(15),
    marker=dict(color='rgb(53, 183, 85)'),
    name = 'Both Sexes'
)

male = go.Bar(
    x = rate2010['Country'].head(15),
    y = rate2010['Male suicide rate'].head(15),
    marker=dict(color='rgb(26, 118, 255)'),
    name = 'Male'
)

female = go.Bar(
    x = rate2010['Country'].head(15),
    y = rate2010['Female suicide rate'].head(15),
    marker=dict(color='rgb(255, 25, 64)'),
    name = 'Female'
)

data = [total, male, female]

layout = go.Layout(
    title='Top countries with the highest suicide rates in 2010',
    xaxis={'title':'Countries'},
    yaxis={'title':'Suicide rate (per 100,000 people)'},
    barmode='group',
    bargap=0.15,
    bargroupgap=0.1
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='basic-bar')

**Countries with highest suicide rates in 2005**

In [ ]:
total = go.Bar(
    x = rate2005['Country'].head(15),
    y = rate2005['Total suicide rate'].head(15),
    marker=dict(color='rgb(53, 183, 85)'),
    name = 'Both Sexes'
)

male = go.Bar(
    x = rate2005['Country'].head(15),
    y = rate2005['Male suicide rate'].head(15),
    marker=dict(color='rgb(26, 118, 255)'),
    name = 'Male'
)

female = go.Bar(
    x = rate2005['Country'].head(15),
    y = rate2005['Female suicide rate'].head(15),
    marker=dict(color='rgb(255, 25, 64)'),
    name = 'Female'
)

data = [total, male, female]

layout = go.Layout(
    title='Top countries with the highest suicide rates in 2005',
    xaxis={'title':'Countries'},
    yaxis={'title':'Suicide rate (per 100,000 people)'},
    barmode='group',
    bargap=0.15,
    bargroupgap=0.1
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='basic-bar')

**Countries with highest suicide rates in 2000**

In [ ]:
total = go.Bar(
    x = rate2000['Country'].head(15),
    y = rate2000['Total suicide rate'].head(15),
    marker=dict(color='rgb(53, 183, 85)'),
    name = 'Both Sexes'
)

male = go.Bar(
    x = rate2000['Country'].head(15),
    y = rate2000['Male suicide rate'].head(15),
    marker=dict(color='rgb(26, 118, 255)'),
    name = 'Male'
)

female = go.Bar(
    x = rate2000['Country'].head(15),
    y = rate2000['Female suicide rate'].head(15),
    marker=dict(color='rgb(255, 25, 64)'),
    name = 'Female'
)

data = [total, male, female]

layout = go.Layout(
    title='Top countries with the highest suicide rates in 2000',
    xaxis={'title':'Countries'},
    yaxis={'title':'Suicide rate (per 100,000 people)'},
    barmode='group',
    bargap=0.15,
    bargroupgap=0.1
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='basic-bar')

### <a id='5.4.2'>5.4.2 Highest Suicide Rates in 2015</a>

In [ ]:
total2015['Code'] = ""
for i in range(len(total2015)):
    country = total2015.iloc[i]['Country']
    if country in countriesCodes.keys():
        total2015['Code'].iloc[i] = countriesCodes[country]
    else:
        total2015['Code'].iloc[i] = np.nan
total2015 = total2015.dropna()

In [ ]:
male2015['Code'] = ""
for i in range(len(male2015)):
    country = male2015.iloc[i]['Country']
    if country in countriesCodes.keys():
        male2015['Code'].iloc[i] = countriesCodes[country]
    else:
        male2015['Code'].iloc[i] = np.nan
male2015 = male2015.dropna()

In [ ]:
female2015['Code'] = ""
for i in range(len(male2015)):
    country = female2015.iloc[i]['Country']
    if country in countriesCodes.keys():
        female2015['Code'].iloc[i] = countriesCodes[country]
    else:
        female2015['Code'].iloc[i] = np.nan
female2015 = female2015.dropna()

**Countries with the highest suicide rates in 2015 for both sexes**

In [ ]:
data = [go.Bar(
    x = total2015['Country'].head(30),
    y = total2015['Total suicide rate'].head(30),
    marker=dict(color='rgb(255, 210, 119)', line=dict(color='rgb(206, 142, 12)', width=1.5)),opacity=0.6 
)]

layout = go.Layout(
    title='Top countries with the highest suicide rates in 2015 for both sexes',
    xaxis={'title':'Countries'},
    yaxis={'title':'Total suicide rate (per 100,000 people)'}
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='basic-bar')

**Top 5 Countries with the highest suicide rates for both sexes in 2015:**
* Sri Lanka - 35.3
* Lithuania - 32.7
* Guyana - 29
* Mongolia - 28.3
* South Korea - 28.3

**Countries with the lowest suicide rates in 2015 for both sexes**

In [ ]:
data = [go.Bar(
    x = total2015['Country'].tail(30),
    y = total2015['Total suicide rate'].tail(30),
    marker=dict(color='rgb(198, 141, 227)', line=dict(color='rgb(127, 20, 181)', width=1.5)),opacity=0.6 
)]

layout = go.Layout(
    title='Top countries with the lowest suicide rates in 2015 for both sexes',
    xaxis={'title':'Countries'},
    yaxis={'title':'Total suicide rate (per 100,000 people)'}
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='basic-bar')

**Top 5 Countries with the lowest suicide rates for both sexes in 2015:**
* Barbados - 0.4
* Grenda - 0.5
* Brunei - 1.3
* Jamaica - 1.4
* The Bahamas - 1.8

**A choropleth map visualizes the suicide rates in 2015 for both sexes throughout the world**

In [ ]:
data = [ dict(
        type = 'choropleth',
        locations = total2015['Code'],
        z = total2015['Total suicide rate'],
        text = total2015['Country'],
        colorscale = [[0,"rgb(255, 60, 20)"],[0.25,"rgb(255, 100, 20)"],[0.5,"rgb(255, 140, 20)"],[0.75,"rgb(255, 200, 20)"],[1,"rgb(225, 255, 150)"]],
        autocolorscale = False,
        reversescale = True,
        marker = dict(
            line = dict (
                color = 'rgb(240, 240, 240)',
                width = 1.5
            ) ),
        colorbar = dict(
            autotick = False,
            title = 'Suicide rate (per 100K)'),
      ) ]

layout = dict(
    title = 'Suicide rates in 2015 - Both sexes',
    geo = dict(
        showframe = False,
        showcoastlines = False,
        projection = dict(
            type = 'Mercator'
        )
    )
    
)

fig = dict( data=data, layout=layout )
py.iplot( fig, validate=False, filename='world-countries.json')

**Countries with the highest suicide rates in 2015 for males**

In [ ]:
data = [go.Bar(
    x = male2015['Country'].head(30),
    y = male2015['Male suicide rate'].head(30),
    marker=dict(color='rgb(126, 229, 212)', line=dict(color='rgb(12, 104, 89)', width=1.5)),opacity=0.6 
)]

layout = go.Layout(
    title='Top countries with the highest suicide rates in 2015 for males',
    xaxis={'title':'Countries'},
    yaxis={'title':'Male suicide rate (per 100,000 people)'}
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='basic-bar')

**Top 5 Countries with the highest suicide rates for males in 2015:**
* Sri Lanka - 58.7
* Lithuania - 58
* Mongolia - 47.8
* Kazakhstan - 46.8
* Guyana - 42.4

**A choropleth map visualizes the suicide rates in 2015 for males in countries throughout the world**

In [ ]:
data = [ dict(
        type = 'choropleth',
        locations = male2015['Code'],
        z = male2015['Male suicide rate'],
        text = male2015['Country'],
        colorscale = [[0,"rgb(255, 60, 20)"],[0.25,"rgb(255, 100, 20)"],[0.5,"rgb(255, 140, 20)"],[0.75,"rgb(255, 200, 20)"],[1,"rgb(225, 255, 150)"]],
        autocolorscale = False,
        reversescale = True,
        marker = dict(
            line = dict (
                color = 'rgb(240, 240, 240)',
                width = 1.5
            ) ),
        colorbar = dict(
            autotick = False,
            title = 'Suicide rate (per 100K)'),
      ) ]

layout = dict(
    title = 'Suicide rates in 2015 - Males',
    geo = dict(
        showframe = False,
        showcoastlines = False,
        projection = dict(
            type = 'Mercator'
        )
    )
    
)

fig = dict( data=data, layout=layout )
py.iplot( fig, validate=False, filename='world-countries.json')

**Countries with the highest suicide rates in 2015 for females**

In [ ]:
data = [go.Bar(
    x = female2015['Country'].head(30),
    y = female2015['Female suicide rate'].head(30),
    marker=dict(color='rgb(255, 165, 210)', line=dict(color='rgb(150, 24, 87)', width=1.5)),opacity=0.6 
)]

layout = go.Layout(
    title='Top countries with the highest suicide rates in 2015 for females',
    xaxis={'title':'Countries'},
    yaxis={'title':'Female suicide rate (per 100,000 people)'}
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='basic-bar')

**Top 5 Countries with the highest suicide rates for females in 2015:**
* South Korea - 16.4
* Guyana - 15.5
* North Korea - 15.4
* India - 14.3
* Sri Lanka - 13.6

We see that the highest males suicide rates can reach up to 4 times the highest females suicide rates.

**A choropleth map visualizes the suicide rates in 2015 for females in countries throughout the world**

In [ ]:
data = [ dict(
        type = 'choropleth',
        locations = female2015['Code'],
        z = female2015['Female suicide rate'],
        text = female2015['Country'],
        colorscale = [[0,"rgb(255, 60, 20)"],[0.25,"rgb(255, 100, 20)"],[0.5,"rgb(255, 140, 20)"],[0.75,"rgb(255, 200, 20)"],[1,"rgb(225, 255, 150)"]],
        autocolorscale = False,
        reversescale = True,
        marker = dict(
            line = dict (
                color = 'rgb(240, 240, 240)',
                width = 1.5
            ) ),
        colorbar = dict(
            autotick = False,
            title = 'Suicide rate (per 100K)'),
      ) ]

layout = dict(
    title = 'Suicide rates in 2015 - Females',
    geo = dict(
        showframe = False,
        showcoastlines = False,
        projection = dict(
            type = 'Mercator'
        )
    )
    
)

fig = dict( data=data, layout=layout )
py.iplot( fig, validate=False, filename='world-countries.json')

### <a id='5.4.3'>5.4.3 Top countries with the highet suicide rates average since the 90's</a>

In [ ]:
totalSince1990 = suicides_merge[maskSince1990][['Country', 'Total suicide rate']].groupby('Country', as_index=False).mean().sort_values(by='Total suicide rate', ascending=False)
mapSince1990 = suicides_merge[maskSince1990][['Code', 'Total suicide rate']].groupby('Code', as_index=False).mean()

In [ ]:
mapSince1990['Country'] = ""
for i in range(len(mapSince1990)):
    mapSince1990['Country'].iloc[i] = countriesNames[mapSince1990.iloc[i]['Code']]

**Countries with the highest suicide rates average since the 90's - both sexes**

In [ ]:
data = [go.Bar(
    x = totalSince1990['Country'].head(30),
    y = totalSince1990['Total suicide rate'].head(30),
    marker=dict(color='rgb(255, 210, 119)', line=dict(color='rgb(206, 142, 12)', width=1.5)),opacity=0.6 
)]

layout = go.Layout(
    title='Top countries with the highest suicide rates average since the 90\'s for both sexes',
    xaxis={'title':'Countries'},
    yaxis={'title':'Total suicide rate (per 100,000 people)'}
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='basic-bar')

**Top 5 Countries with the highest suicide rates average since the 90's for both sexes:**
* Lithuania - 38.32
* Guyana - 35.75
* Russia - 33
* Somalia - 29.86
* Latvia - 29

*(The average is calculated only for years for which data was provided)*

**Countries with the lowest suicide rates average since the 90's - both sexes**

In [ ]:
data = [go.Bar(
    x = totalSince1990['Country'].tail(30),
    y = totalSince1990['Total suicide rate'].tail(30),
    marker=dict(color='rgb(198, 141, 227)', line=dict(color='rgb(127, 20, 181)', width=1.5)),opacity=0.6 
)]

layout = go.Layout(
    title='Top countries with the lowest suicide rates average since the 90\'s for both sexes',
    xaxis={'title':'Countries'},
    yaxis={'title':'Total suicide rate (per 100,000 people)'}
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='basic-bar')

**Top 5 Countries with the lowest suicide rates average since the 90's for both sexes:**
* Jordan - 0.2
* Syria - 0.52
* Cyprus - 0.61
* Egypt - 0.94
* Brunei - 1.37

*(The average is calculated only for years for which data was provided)*

**A choropleth map visualizes the suicide rates average since the 90's in countries throughout the world**

In [ ]:
data = [ dict(
        type = 'choropleth',
        locations = mapSince1990['Code'],
        z = mapSince1990['Total suicide rate'],
        text = mapSince1990['Country'],
        colorscale = [[0,"rgb(255, 60, 20)"],[0.25,"rgb(255, 100, 20)"],[0.5,"rgb(255, 140, 20)"],[0.75,"rgb(255, 200, 20)"],[1,"rgb(225, 255, 150)"]],
        autocolorscale = False,
        reversescale = True,
        marker = dict(
            line = dict (
                color = 'rgb(240, 240, 240)',
                width = 1.5
            ) ),
        colorbar = dict(
            autotick = False,
            title = 'Suicide rate (per 100K)'),
      ) ]

layout = dict(
    title = 'Suicide rates average since the 90\'s',
    geo = dict(
        showframe = False,
        showcoastlines = False,
        projection = dict(
            type = 'Mercator'
        )
    )
    
)

fig = dict( data=data, layout=layout )
py.iplot( fig, validate=False, filename='world-countries.json')

## <a id='5.5'>5.5 Top countries with the highest or lowest male-female ratio of suicide rates</a>

In [ ]:
ratio2000 = suicides[mask2000][['Country', 'Male-Female ratio']].sort_values(by='Male-Female ratio', ascending=False)
ratio2005 = suicides[mask2005][['Country', 'Male-Female ratio']].sort_values(by='Male-Female ratio', ascending=False)
ratio2010 = suicides[mask2010][['Country', 'Male-Female ratio']].sort_values(by='Male-Female ratio', ascending=False)
ratio2015 = suicides[mask2015][['Country', 'Male-Female ratio']].sort_values(by='Male-Female ratio', ascending=False)

In [ ]:
ratio2000 = ratio2000.replace([0, np.inf], np.nan)
ratio2005 = ratio2005.replace([0, np.inf], np.nan)
ratio2010 = ratio2010.replace([0, np.inf], np.nan)
ratio2015 = ratio2015.replace([0, np.inf], np.nan)

In [ ]:
ratio2000 = ratio2000.dropna()
ratio2005 = ratio2005.dropna()
ratio2010 = ratio2010.dropna()
ratio2015 = ratio2015.dropna()

**Male-Female suicides ratio in 2015**

In [ ]:
data = [go.Bar(
    x = ratio2015['Country'].head(30),
    y = ratio2015['Male-Female ratio'].head(30),
    marker=dict(color='rgb(255, 210, 119)', line=dict(color='rgb(206, 142, 12)', width=1.5)),opacity=0.6 
)]

layout = go.Layout(
    title='Top countries with the highest male-female suicides ratio in 2015',
    xaxis={'title':'Countries'},
    yaxis={'title':'Male-Female suicide rate ratio'}
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='basic-bar')

**Top 5 Countries with the highest male-female suicides ratio in 2015:**
* Saint Vincer - 12.25
* Qatar - 9.44
* Panama - 9
* Jamaica - 8.66
* The Bahamas - 8

In [ ]:
data = [go.Bar(
    x = ratio2015['Country'].tail(30),
    y = ratio2015['Male-Female ratio'].tail(30),
    marker=dict(color='rgb(198, 141, 227)', line=dict(color='rgb(127, 20, 181)', width=1.5)),opacity=0.6 
)]

layout = go.Layout(
    title='Top countries with the lowest male-female suicides ratio in 2015',
    xaxis={'title':'Countries'},
    yaxis={'title':'Male-Female suicide rate ratio'}
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='basic-bar')

**Top 5 Countries with the lowest male-female suicides ratio in 2015:**
* Bangladesh - 0.7 (Meaning the female-male ratio is 1.42)
* China - 0.756 (Female-male ratio is 1.32)
* North Korea - 1.06
* Pakistan - 1.1
* Nepal - 1.18

We see that only two countries in the world have a male-female ratio which is below 1, meaning that the female suicide rate is higher than the male suicide rate.

We also see that North Korea has one of the lowest male-female ratio in the world. We've seen earlier that North Korea also has one of the highest females suicide rate in the world.

**Male-Female suicides ratio in 2010**

In [ ]:
data = [go.Bar(
    x = ratio2010['Country'].head(30),
    y = ratio2010['Male-Female ratio'].head(30),
    marker=dict(color='rgb(255, 210, 119)', line=dict(color='rgb(206, 142, 12)', width=1.5)),opacity=0.6 
)]

layout = go.Layout(
    title='Top countries with the highest male-female suicides ratio in 2010',
    xaxis={'title':'Countries'},
    yaxis={'title':'Male-Female suicide rate ratio'}
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='basic-bar')

**Top 5 Countries with the highest male-female suicides ratio in 2010:**
* Qatar - 9
* Saint Vincer - 8.5
* Seychelles - 8
* Malta - 7.82
* Rwanda - 7.21

In [ ]:
data = [go.Bar(
    x = ratio2010['Country'].tail(30),
    y = ratio2010['Male-Female ratio'].tail(30),
    marker=dict(color='rgb(198, 141, 227)', line=dict(color='rgb(127, 20, 181)', width=1.5)),opacity=0.6 
)]

layout = go.Layout(
    title='Top countries with the lowest male-female suicides ratio in 2010',
    xaxis={'title':'Countries'},
    yaxis={'title':'Male-Female suicide rate ratio'}
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='basic-bar')

**Top 5 Countries with the lowest male-female suicides ratio in 2010:**
* Bangladesh - 0.53 (Female-male ratio is 1.89)
* China - 0.75 (Female-male ratio is 1.33)
* Pakistan - 1
* North Korea - 1.02
* India - 1.125

Pakistan's male-female suicide rate ratio is 1. That means the males suicide rate and the females suicide rate are roughly equal.

**Male-Female suicides ratio in 2005**

In [ ]:
data = [go.Bar(
    x = ratio2005['Country'].head(30),
    y = ratio2005['Male-Female ratio'].head(30),
    marker=dict(color='rgb(255, 210, 119)', line=dict(color='rgb(206, 142, 12)', width=1.5)),opacity=0.6 
)]

layout = go.Layout(
    title='Top countries with the highest male-female suicides ratio in 2005',
    xaxis={'title':'Countries'},
    yaxis={'title':'Male-Female suicide rate ratio'}
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='basic-bar')

**Top 5 Countries with the highest male-female suicides ratio in 2005:**
* Barbados - 9
* Belize - 8.18
* Seychelles - 7.95
* Rwanda - 7.92
* Grenada - 7.66

In [ ]:
data = [go.Bar(
    x = ratio2005['Country'].tail(30),
    y = ratio2005['Male-Female ratio'].tail(30),
    marker=dict(color='rgb(198, 141, 227)', line=dict(color='rgb(127, 20, 181)', width=1.5)),opacity=0.6 
)]

layout = go.Layout(
    title='Top countries with the lowest male-female suicides ratio in 2005',
    xaxis={'title':'Countries'},
    yaxis={'title':'Male-Female suicide rate ratio'}
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='basic-bar')

**Top 5 Countries with the lowest male-female suicides ratio in 2005:**
* Bangladesh - 0.5 (Female-male ratio is 2)
* China - 0.754 (Female-male ratio is 1.32)
* North Korea - 1.03
* India - 1.09
* Nepal - 1.12

**Male-Female suicides ratio in 2000**

In [ ]:
data = [go.Bar(
    x = ratio2000['Country'].head(30),
    y = ratio2000['Male-Female ratio'].head(30),
    marker=dict(color='rgb(255, 210, 119)', line=dict(color='rgb(206, 142, 12)', width=1.5)),opacity=0.6 
)]

layout = go.Layout(
    title='Top countries with the highest male-female suicides ratio in 2000',
    xaxis={'title':'Countries'},
    yaxis={'title':'Male-Female suicide rate ratio'}
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='basic-bar')

**Top 5 Countries with the highest male-female suicides ratio in 2000:**
* Antigua and Barbuda - 49
* Saint Vincent and the Grenadines - 17
* Rwanda - 9.4
* Venezuela - 7.61
* Seychelles - 7.6

In [ ]:
data = [go.Bar(
    x = ratio2000['Country'].tail(30),
    y = ratio2000['Male-Female ratio'].tail(30),
    marker=dict(color='rgb(198, 141, 227)', line=dict(color='rgb(127, 20, 181)', width=1.5)),opacity=0.6 
)]

layout = go.Layout(
    title='Top countries with the lowest male-female suicides ratio in 2000',
    xaxis={'title':'Countries'},
    yaxis={'title':'Male-Female suicide rate ratio'}
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='basic-bar')

**Top 5 Countries with the lowest male-female suicides ratio in 2000:**
* Bangladesh - 0.72 (Female-male ratio is 2)
* China - 0.77 (Female-male ratio is 1.32)
* Tonga - 1.11
* India - 1.14
* Nepal - 1.16

## <a id='5.6'>5.6 Further analysis on particular countries</a>

After analyzing the top countries with the highest rates in the world, we will now look into particular countries which seem interesting after the analysis we made.

Lithuania, Sri Lanka and Belarus have very high suicide rates generally. North Korea has relatively an exetremely low ratio between men and women but high female suicide rate. Japan and South Korea are both significant and large western countries with high suicidres in relation to other nations. Belguim is a European country with high suicide rates, particularly for women, and Poland is an East-European Country with high suicide rates, particularly for men.

Thus, the selected countries for further analysis are:

* Lithuania
* Sri Lanka
* Belarus
* North Korea
* Japan
* South Korea
* Poland
* Belguim

We will use scatter graphs to present the rates of each country in the years 2000, 2005, 2010 and 2015. The data of each country will be presented as a line connecting the points which contain the data for each year, so we could either look into each country seperately or compare between the countries, by a single year or by the total trend.

In [ ]:
selected = ['Lithuania', 'Sri Lanka', 'Belarus', 'North Korea', 'Japan', 'South Korea', 'Poland', 'Belguim']
selectedMasks = []
for country in selected:
    mask = suicides['Country'].str.contains(country)
    selectedMasks.append(mask)

**Total suicide rates of the selected countries**

In [ ]:
data = []
for i in range(len(selected)):
    
    trace = go.Scatter(
        x = ['2000', '2005', '2010', '2015'],
        y = suicides[selectedMasks[i]]['Total suicide rate'].values.tolist(),
        name = selected[i],
        line=dict(
            shape='spline'
        )
    )
    data.append(trace)
    
layout = go.Layout(
    title='Total Suicide Rates of the Selected Countries',
    xaxis=dict(
        title='Year',
        autotick=False,
        ticks='outside',
        tick0=0,
        dtick=5
    ),
    yaxis=dict(
        title='Total Suicide Rate',
        rangemode='tozero',
        dtick=5
    )
)

fig = go.Figure(data=data, layout=layout)

py.iplot(fig, filename='line-mode')

**Males suicide rates of the selected countries**

In [ ]:
data = []
for i in range(len(selected)):
    
    trace = go.Scatter(
        x = ['2000', '2005', '2010', '2015'],
        y = suicides[selectedMasks[i]]['Male suicide rate'].values.tolist(),
        name = selected[i],
        line=dict(
            shape='spline'
        )
    )
    data.append(trace)
    
layout = go.Layout(
    title='Males Suicide Rates of the Selected Countries',
    xaxis=dict(
        title='Year',
        autotick=False,
        ticks='outside',
        tick0=0,
        dtick=5
    ),
    yaxis=dict(
        title='Males Suicide Rate',
        rangemode='tozero'
    )
)

fig = go.Figure(data=data, layout=layout)

py.iplot(fig, filename='line-mode')

**Females suicide rates of the selected countries**

In [ ]:
data = []
for i in range(len(selected)):
    
    trace = go.Scatter(
        x = ['2000', '2005', '2010', '2015'],
        y = suicides[selectedMasks[i]]['Female suicide rate'].values.tolist(),
        name = selected[i],
        line=dict(
            shape='spline'
        )
    )  
    data.append(trace)
  
layout = go.Layout(
    title='Females Suicide Rates of the Selected Countries',
    xaxis=dict(
        title='Year',
        autotick=False,
        ticks='outside',
        tick0=0,
        dtick=5
    ),
    yaxis=dict(
        title='Females Suicide Rate',
        rangemode='tozero'
    )
)

fig = go.Figure(data=data, layout=layout)

py.iplot(fig, filename='line-mode')

**Male-Female suicide rates ratio in the selected countries**

In [ ]:
data = []
for i in range(len(selected)):
    
    trace = go.Scatter(
        x = ['2000', '2005', '2010', '2015'],
        y = suicides[selectedMasks[i]]['Male-Female ratio'].values.tolist(),
        name = selected[i],
        line=dict(
            shape='spline'
        )
    )
    data.append(trace)

layout = go.Layout(
    title='Male-Female Suicide Rates Ratio in the Selected Countries',
    xaxis=dict(
        title='Year',
        autotick=False,
        ticks='outside',
        tick0=0,
        dtick=5
    ),
    yaxis=dict(
        title='Male-Female Ratio',
        rangemode='tozero'
    )
)

fig = go.Figure(data=data, layout=layout)

py.iplot(fig, filename='line-mode')

# <a id='6'>6. Summary and Conculsions</a>

## <a id='6.1'>6.1 Summary of the data presented in the notebook</a>

**Top 10 countries with the highest suicide rates for both sexes:**

*(Top 10 in 2015, although data is shown for 2000, 2005, 2010 and 2015 and ordered by the data from 2000)*

* Lithuania
* Belarus
* Kazakhstan
* Sri Lanka
* Mongolia
* Suriname
* Equatorial Guinea
* Guyana
* Poland
* South Korea

**Top 10 countries with the highest suicide rates for males:**

*(Top 10 in 2015, although data is shown for 2000, 2005, 2010 and 2015 and ordered by the data from 2000)*

* Lithuania
* Belarus
* Kazakhstan
* Latvia
* Sri Lanka
* Mongolia
* Suriname
* Poland
* Guyana
* South Korea

**Top 10 countries with the highest suicide rates for females:**

*(Top 10 in 2015, although data is shown for 2000, 2005, 2010 and 2015 and ordered by the data from 2000)*

* Sri Lanka
* North Korea
* India
* Bolivia
* Japan
* Belguim
* Guyana
* Suriname
* Angola
* South Korea

**Top 5 Countries with the highest suicide rates for both sexes in 2015:**
* Sri Lanka - 35.3
* Lithuania - 32.7
* Guyana - 29
* Mongolia - 28.3
* South Korea - 28.3

**Top 5 Countries with the lowest suicide rates for both sexes in 2015:**
* Barbados - 0.4
* Grenda - 0.5
* Brunei - 1.3
* Jamaica - 1.4
* The Bahamas - 1.8

**Top 5 Countries with the highest suicide rates for males in 2015:**
* Sri Lanka - 58.7
* Lithuania - 58
* Mongolia - 47.8
* Kazakhstan - 46.8
* Guyana - 42.4

**Top 5 Countries with the highest suicide rates for females in 2015:**
* South Korea - 16.4
* Guyana - 15.5
* North Korea - 15.4
* India - 14.3
* Sri Lanka - 13.6



**Top 5 Countries with the highest suicide rates average since the 90's for both sexes:**

*(The average is calculated only for years for which data was provided)*

* Lithuania - 38.32
* Guyana - 35.75
* Russia - 33
* Somalia - 29.86
* Latvia - 29

**Top 5 Countries with the lowest suicide rates average since the 90's for both sexes:**

*(The average is calculated only for years for which data was provided)*

* Jordan - 0.2
* Syria - 0.52
* Cyprus - 0.61
* Egypt - 0.94
* Brunei - 1.37

**Top 5 Countries with the highest male-female suicides ratio in 2015:**
* Saint Vincer - 12.25
* Qatar - 9.44
* Panama - 9
* Jamaica - 8.66
* The Bahamas - 8

**Top 5 Countries with the lowest male-female suicides ratio in 2015:**
* Bangladesh - 0.7 (Meaning the female-male ratio is 1.42)
* China - 0.756 (Female-male ratio is 1.32)
* North Korea - 1.06
* Pakistan - 1.1
* Nepal - 1.18

**Top 5 Countries with the highest male-female suicides ratio in 2010:**
* Qatar - 9
* Saint Vincer - 8.5
* Seychelles - 8
* Malta - 7.82
* Rwanda - 7.21

**Top 5 Countries with the lowest male-female suicides ratio in 2010:**
* Bangladesh - 0.53 (Female-male ratio is 1.89)
* China - 0.75 (Female-male ratio is 1.33)
* Pakistan - 1
* North Korea - 1.02
* India - 1.125



**Top 5 Countries with the highest male-female suicides ratio in 2005:**
* Barbados - 9
* Belize - 8.18
* Seychelles - 7.95
* Rwanda - 7.92
* Grenada - 7.66

**Top 5 Countries with the lowest male-female suicides ratio in 2005:**
* Bangladesh - 0.5 (Female-male ratio is 2)
* China - 0.754 (Female-male ratio is 1.32)
* North Korea - 1.03
* India - 1.09
* Nepal - 1.12

**Top 5 Countries with the highest male-female suicides ratio in 2000:**
* Antigua and Barbuda - 49
* Saint Vincent and the Grenadines - 17
* Rwanda - 9.4
* Venezuela - 7.61
* Seychelles - 7.6

**Top 5 Countries with the lowest male-female suicides ratio in 2000:**
* Bangladesh - 0.72 (Female-male ratio is 2)
* China - 0.77 (Female-male ratio is 1.32)
* Tonga - 1.11
* India - 1.14
* Nepal - 1.16

## <a id='6.2'>6.2 Conclusions</a>

* The highest males suicide rates in specific countries can reach up to 4 times the highest females suicide rates in specific countries.

* Only two countries in the world have a male-female suicide rates ratio which is below 1, meaning that the female suicide rate is higher than the male suicide rate.

* North Korea has one of the lowest male-female ratio in the world but also has one of the highest females suicide rate in the world.

* Pakistan's male-female suicide rate ratio in 2010 was 1. That means the males suicide rate and the females suicide rate were roughly equal.

**It is highly recommended to look through the data in the specific countries we've gone through in section 5.6.**

## Thanks for reading the notebook!